In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedGroupKFold, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import seaborn as sns
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, accuracy_score, recall_score, f1_score, precision_score
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
import joblib

os.chdir('..')
print(os.getcwd())

### CSV loading

In [ ]:
# # df = pd.read_csv('opensmile.csv')
# df = pd.read_csv('opensmile_standardized.csv')
# # df = shuffle(df)
# # df.reset_index(inplace=True, drop=True)
# # df.drop('voiceID', inplace = True, axis = 1)
# df['label'].value_counts()
# df['gender'].value_counts()
# df['age'].value_counts()
# df

In [ ]:
# x = df.iloc[:, 4:].values  # feather
# y = df['label'].values  # label
# speakers = df['speaker'].values  # Speaker information

# kf = StratifiedGroupKFold(n_splits=5, shuffle=True)

# X_train_folds = []
# Y_train_folds = []
# X_test_folds = []
# Y_test_folds = []
# speaker_train_folds = []  # Used to store the speaker corresponding to each training fold
# speaker_test_folds = []  # Used to store the speaker corresponding to each test fold

# for train_index, test_index in kf.split(x, y, speakers):
#     X_train_folds.append(x[train_index])
#     Y_train_folds.append(y[train_index])
#     X_test_folds.append(x[test_index])
#     Y_test_folds.append(y[test_index])
    
#     # Save the speaker information for each compromise
#     speaker_train_folds.append(speakers[train_index])
#     speaker_test_folds.append(speakers[test_index])

# # Save Data
# for i in range(5):
#     # Save the training and testing datase
#     np.save(f'./X_train_{i}.npy', X_train_folds[i])
#     np.save(f'./Y_train_{i}.npy', Y_train_folds[i])
#     np.save(f'./X_test_{i}.npy', X_test_folds[i])
#     np.save(f'./Y_test_{i}.npy', Y_test_folds[i])
    
#     # Save the speaker information for each fold
#     # np.save(f'./speaker_train_{i}.npy', speaker_train_folds[i])
#     # np.save(f'./speaker_test_{i}.npy', speaker_test_folds[i])
#     # Save the speaker information for each fold to a txt file
#     with open(f'./speaker_train_{i}.txt', 'w') as f_train:
#         for speaker in speaker_train_folds[i]:
#             f_train.write(f'{speaker}\n')  # Write one line for each speaker's information
    
#     with open(f'./speaker_test_{i}.txt', 'w') as f_test:
#         for speaker in speaker_test_folds[i]:
#             f_test.write(f'{speaker}\n')  # Write one line for each speaker's information



In [ ]:
X_train = np.load('./X_all_train_mutual.npy')
Y_train = np.load('./Y_train.npy')
X_test = np.load('./X_all_test_mutual.npy')
Y_test = np.load('./Y_test.npy')

X_train, Y_train = shuffle(X_train, Y_train, random_state=42)
X_test, Y_test = shuffle(X_test, Y_test, random_state=42)


### Feature scaling

In [ ]:
# # Scale
# sc = StandardScaler()

# # sc = MinMaxScaler()
# # sc = joblib.load(scaler.pkl")

# # sc = RobustScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

# # Save data scale parameters
# joblib.dump(sc, "scaler.pkl")

# # pd.DataFrame(X_test)

# KNN

### Hyperparameter tuning for KNN

In [ ]:
# Set hyperparameters
leaf_size = list(range(1, 50))
n_neighbors = list(range(1, 20))
p = [1, 2]
knn_hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

# Define KNN model
knn = KNeighborsClassifier()

# Use the existing cross-validation (do not set cv again in GridSearchCV)
knn_clf = GridSearchCV(knn, knn_hyperparameters)  # Remove cv=10

# fitted model
knn_best_model = knn_clf.fit(X_train, Y_train)

# Save the best model
joblib.dump(knn_best_model, "./knn.pkl")

# Output the optimal hyperparameters and scores
print('Best leaf_size:       ', knn_best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:               ', knn_best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:     ', knn_best_model.best_estimator_.get_params()['n_neighbors'])
print('Best Score:           %s' % knn_best_model.best_score_)
print('Best Hyperparameters: %s' % knn_best_model.best_params_)

# Prediction test set
y_pred_knn = knn_best_model.predict(X_test)

# Calculate confusion matrix and performance metrics
knn_conf_matrix = confusion_matrix(Y_test, y_pred_knn)
tn, fp, fn, tp = knn_conf_matrix.ravel()
knn_sen = tp / (tp + fn)
knn_spe = tn / (tn + fp)

# result
print(f"ConfusionMatrix: \n {knn_conf_matrix}")      
print('Accuracy: ', accuracy_score(Y_test, y_pred_knn))  # Accuracy
print(f'Specificity: {knn_spe}')  # Specificity
print(f'Sensitivity:  {knn_sen}')  # Sensitivity
print('F1-score: ', f1_score(Y_test, y_pred_knn, average='binary'))  # F1-score
print('Recall: ', recall_score(Y_test, y_pred_knn, average='binary'))  # Recall
print('Precision: ', precision_score(Y_test, y_pred_knn, average='binary'))  # Precision

# Decision Trees

### Hyperparameter tuning for Decision Trees

In [ ]:
# Set hyperparameters
max_depth = list(range(1, 10))
min_samples_split = list(range(2, 10))
min_samples_leaf = list(range(1, 5))
criterion = ['gini', 'entropy']
dt_hyperparameters = dict(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, criterion=criterion)

# Define decision tree model
dt = tree.DecisionTreeClassifier()

# Using GridSearchCV for hyperparameter selection does not require specifying the cv parameter
dt_clf = GridSearchCV(dt, dt_hyperparameters, refit=True)  # Remove cv=10

# fitted model
dt_best_model = dt_clf.fit(X_train, Y_train)

# Save the best model
joblib.dump(dt_best_model, "dt.pkl")

# Output the optimal hyperparameters and scores
print('Best Score:           %s' % dt_best_model.best_score_)
print('Best Hyperparameters: %s' % dt_best_model.best_params_)

# Prediction test set
y_pred_dt = dt_best_model.predict(X_test)

# Calculate confusion matrix and performance metrics
dt_conf_matrix = confusion_matrix(Y_test, y_pred_dt)
tn, fp, fn, tp = dt_conf_matrix.ravel()
dt_sen = tp / (tp + fn)
dt_spe = tn / (tn + fp)

# result
print(f"ConfusionMatrix: \n {dt_conf_matrix}")      
print('Accuracy:  ', accuracy_score(Y_test, y_pred_dt))  # Accuracy
print(f'Specificity: {dt_spe}')  # Specificity
print(f'Sensitivity:  {dt_sen}')  # Sensitivity
print('F1-score: ', f1_score(Y_test, y_pred_dt, average='binary'))  # F1-score
print('Recall: ', recall_score(Y_test, y_pred_dt, average='binary'))  # Recall
print('Precision: ', precision_score(Y_test, y_pred_dt, average='binary'))  # Precision

# SVM

### Hyperparameter tuning for SVM

In [ ]:
# Set hyperparameters
C = [0.1, 1, 10, 100, 1000]
gamma = [1, 0.1, 0.01, 0.001, 0.0001]
# rbf -> Gaussian kernel, poly -> polynomial kernel, linear -> linear kernel, sigmoid -> sigmoid kernel
kernel = ['rbf', 'poly', 'linear', 'sigmoid']
svm_hyperparameters = dict(C=C, gamma=gamma, kernel=kernel)

# Define SVM model
svm2 = svm.SVC(probability=True)

# When using GridSearchCV for hyperparameter selection, there is no need to specify the cv parameter
svm_clf = GridSearchCV(svm2, svm_hyperparameters, refit=True)  # Remove cv=10

# fitted model
svm_best_model = svm_clf.fit(X_train, Y_train)

# Save the best model
joblib.dump(svm_best_model, "svm.pkl")

# Output the optimal hyperparameters and scores
print('Best Score:           %s' % svm_best_model.best_score_)
print('Best Hyperparameters: %s' % svm_best_model.best_params_)

# Prediction test set
y_pred_svm = svm_best_model.predict(X_test)

# Calculate confusion matrix and performance metrics
svm_conf_matrix = confusion_matrix(Y_test, y_pred_svm)
tn, fp, fn, tp = svm_conf_matrix.ravel()
svm_sen = tp / (tp + fn)
svm_spe = tn / (tn + fp)

# result
print(f"ConfusionMatrix: \n {svm_conf_matrix}")      
print('Accuracy:  ', accuracy_score(Y_test, y_pred_svm))  # Accuracy
print(f'Specificity: {svm_spe}')  # Specificity
print(f'Sensitivity:  {svm_sen}')  # Sensitivity
print('F1-score: ', f1_score(Y_test, y_pred_svm, average='binary'))  # F1-score
print('Recall: ', recall_score(Y_test, y_pred_svm, average='binary'))  # Recall
print('Precision: ', precision_score(Y_test, y_pred_svm, average='binary'))  # Precision


# AdaBoost

### Hyperparameters tuning for AdaBoost

In [ ]:
# Set hyperparameters
n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 200]
learning_rate = [1, 0.5, 0.25, 0.1, 0.05, 0.01]
abc_hyperparameters = dict(n_estimators=n_estimators, learning_rate=learning_rate)

# Define AdaBoost model
abc_model = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=1))

# Use GridSearchCV for hyperparameter selection, but cv parameter is no longer needed
abc_clf = GridSearchCV(abc_model, abc_hyperparameters, refit=True)  # Remove cv parameters

# fitted model
abc_best_model = abc_clf.fit(X_train, Y_train)

# Save the best model
joblib.dump(abc_best_model, "ab.pkl")

# Output the optimal hyperparameters and scores
print('Best Score:           %s' % abc_best_model.best_score_)
print('Best Hyperparameters: %s' % abc_best_model.best_params_)

# Prediction test set
y_pred_abc = abc_best_model.predict(X_test)

# Calculate confusion matrix and performance metrics
abc_conf_matrix = confusion_matrix(Y_test, y_pred_abc)
tn, fp, fn, tp = abc_conf_matrix.ravel()
abc_sen = tp / (tp + fn)
abc_spe = tn / (tn + fp)

# result
print(f"ConfusionMatrix: \n {abc_conf_matrix}")      
print('Accuracy:  ', accuracy_score(Y_test, y_pred_abc))  # Accuracy
print(f'Specificity: {abc_spe}')  # Specificity
print(f'Sensitivity:  {abc_sen}')  # Sensitivity
print('Recall: ', recall_score(Y_test, y_pred_abc, average='binary'))  # Recall
print('F1-score: ', f1_score(Y_test, y_pred_abc, average='binary'))  # F1-score
print('Precision: ', precision_score(Y_test, y_pred_abc, average='binary'))  # Precision


# Random Forest

### Hyperparameters tuning for RF

In [ ]:
# Set hyperparameters
n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 200]
max_depth = [1, 3, 5, 7, 9]
max_features = ['auto', 'sqrt', 'log2']
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 5, 10, 15]

rf_hyperparameters = dict(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features, 
                          min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

# Create a random forest classifier
rf_model = RandomForestClassifier()

# Use GridSearchCV for hyperparameter selection and remove the cv parameter
rf_clf = GridSearchCV(rf_model, rf_hyperparameters, refit=True)  # Remove cv parameter

# fitted model
rf_best_model = rf_clf.fit(X_train, Y_train)

# Save the best model
joblib.dump(rf_best_model, "rf.pkl")

# Output the optimal hyperparameters and scores
print('Best Score:           %s' % rf_best_model.best_score_)
print('Best Hyperparameters: %s' % rf_best_model.best_params_)

# Prediction test set
y_pred_rf = rf_best_model.predict(X_test)

# Calculate confusion matrix and performance metrics
rf_conf_matrix = confusion_matrix(Y_test, y_pred_rf)
tn, fp, fn, tp = rf_conf_matrix.ravel()
rf_sen = tp / (tp + fn)
rf_spe = tn / (tn + fp)

# result
print(f"ConfusionMatrix: \n {rf_conf_matrix}")      
print('Accuracy:  ', accuracy_score(Y_test, y_pred_rf))  # Accuracy
print(f'Specificity: {rf_spe}')  # Specificity
print(f'Sensitivity:  {rf_sen}')  # Sensitivity
print('Recall: ', recall_score(Y_test, y_pred_rf, average='binary'))  # Recall
print('F1-score: ', f1_score(Y_test, y_pred_rf, average='binary'))  # F1-score
print('Precision: ', precision_score(Y_test, y_pred_rf, average='binary'))  # Precision


In [ ]:
import xgboost as xgb

# Set hyperparameters
n_estimators = [50, 100, 200, 300]
learning_rate = [0.1, 0.01, 0.05, 0.3]
max_depth = [3, 6, 9, 12]
subsample = [0.7, 0.8, 0.9, 1.0]
colsample_bytree = [0.7, 0.8, 1.0]

xgb_hyperparameters = dict(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth,
                           subsample=subsample, colsample_bytree=colsample_bytree)

# Create an XGBoost classifier
xgb_model = xgb.XGBClassifier()

# Use GridSearchCV for hyperparameter selection and remove the cv parameter
xgb_clf = GridSearchCV(xgb_model, xgb_hyperparameters, refit=True)

# fitted model
xgb_best_model = xgb_clf.fit(X_train, Y_train)

# Save the best model
joblib.dump(xgb_best_model, "xgb.pkl")

# Output the optimal hyperparameters and scores
print('Best Score:           %s' % xgb_best_model.best_score_)
print('Best Hyperparameters: %s' % xgb_best_model.best_params_)

# Prediction test set
y_pred_xgb = xgb_best_model.predict(X_test)

# Calculate confusion matrix and performance metrics
xgb_conf_matrix = confusion_matrix(Y_test, y_pred_xgb)
tn, fp, fn, tp = xgb_conf_matrix.ravel()
xgb_sen = tp / (tp + fn)
xgb_spe = tn / (tn + fp)

# result
print(f"ConfusionMatrix: \n {xgb_conf_matrix}")      
print('Accuracy:  ', accuracy_score(Y_test, y_pred_xgb))  # Accuracy
print(f'Specificity: {xgb_spe}')  # Specificity
print(f'Sensitivity:  {xgb_sen}')  # Sensitivity
print('Recall: ', recall_score(Y_test, y_pred_xgb, average='binary'))  # Recall
print('F1-score: ', f1_score(Y_test, y_pred_xgb, average='binary'))  # F1-score
print('Precision: ', precision_score(Y_test, y_pred_xgb, average='binary'))  # Precision
